In [ ]:
from ortools.sat.python import cp_model
# Derived from https://stackoverflow.com/questions/58934609/obtain-list-of-sat-solutions-from-ortools
class VarArraySolutionCollector(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.solution_list = []

    def on_solution_callback(self):
        self.solution_list.append([self.Value(v) for v in self.__variables])
    
    def __repr__(self):
      print(f"There are {len(self.solution_list)} solutions of {len(self.__variables)} variables.")
      true = []
      false = []
      undecided = []
      for i, v in enumerate(self.__variables):
        values = set(l[i] for l in self.solution_list)
        if values == {1}:
          true += [f'{v}']
        elif values == {0}:
          false += [f'{v}']
        elif values == {0, 1}:
          undecided += [f'{v}']
        else:
          raise ValueError(f"WTF {v} {values}")
      answer =  f'*****************\nTrue statements ({len(true)}): \n'
      answer += '\n'.join(true)+'\n'
      answer += f'*****************\nUndecided statements ({len(undecided)}): \n'
      answer += '\n'.join(undecided)+'\n'
      answer += f'*****************\nFalse statements ({len(false)}): \n'
      answer += '\n'.join(false)+'\n'
      return answer


In [ ]:
model = cp_model.CpModel()
wire = model.NewBoolVar("The wire is plugged")
switch = model.NewBoolVar("The switch is closed")
bulb = model.NewBoolVar("The bulb is on")
model.AddImplication(wire and switch, bulb)
model.Add(wire == 0)
model.Add(switch == 1)

solver = cp_model.CpSolver()
solution_collector = VarArraySolutionCollector([wire, switch, bulb])
solver.SearchForAllSolutions(model, solution_collector)
print(solution_collector)